# 1.0 Data Understanding

We had little missing data(based on the count of each feature). We found that only the "Code" feature had missing values. We decided not to interpolate these values, as the code is nominal data, and the result of the ensuing play after each pitch is not something we thought was wise to interpolate. We have more than enough data without these rows anyway.
    
Even with a mostly pristine dataset, we still had some housekeeping items to take care of.
    
* The first bit of data cleanup we did is to get rid of duplicate rows.<br> Even without the unique identifier, there is a sufficient number of features, many of which relate to the state of the ball game, to determine if an entry is a duplicate.<br> The probability of each and every feature being identical for 2 seperate pitches,especially considering some of the data is unrelated to the actual throw,<br> is incredibly low.
        
* The next important step we took to groom out dataset is to remove pitches where the prediction class is listed as unknown, or where it was listed as 'FA'.<br> While the data itself was in great condition as is, the overview written by the data aggregator wasn't quite as outstanding. <br>This value('FA') wasn't found in the key that was provided to translate the pitch_type codes into their real-world values, so we dropped this data.<br> Likewise, we dropped all unknown prediction classes, as it's an incorrect practice to interpolate this data.
        
       
* Next, we removed outliers in the data. It isn't always easy to differentiate true outliers from valuable data points that happen to be unusual.<br> Luckily, some of the features had very obvious outliers well over 10 times the standard deviation apart from the mean. <br> After some tweaking of the parameters,<br> we found that removing any value more than 12 standard deviations away from the mean produced acceptable numbers that make<br> sense in the context of the business case.We enlisted the advice of a subject matter expert to help us decide on what summary statistics look most realistic.
      

* Finally, we removed any data points that the classifiers were not fully confident in. <br>Fortunately, the curators of this data provided a confidence interval that we could use to remove pitches that were 'guesses' of sorts. <br>This is helpful because we can train a prediction model based only on data that is concrete.<br> We also removed any values that were missing in this step, as we have sufficient data to do so, and interpolating this data can be difficult.
    


The dataset consists of 39 attributes and 15 prediction classes describing the information of the pitch thrown. Of these 39 attributes, we have chosen 23 attributes that are most likely candidates to use for the classification task. The important numeric features among these are: <br><br>1. Break length <br>2. Break angle <br>3. Start speed <br>4. End speed <br>5. Spin rate <br>6. Pitch Number <br>7. ax <br> 8. ay <br> 9. az <br> 10. Nasty <br> 11. x <br> 12. y<br><br>
The important categorical features among these are:<br><br> 1. on_1b <br> 2. on_2b <br> 3. on_3b <br> 4. outs <br> 5. s_count and <br> 6. b_count


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

df = pd.read_csv("../pitches.csv")


In [2]:
from sklearn.preprocessing import LabelEncoder
#drop unkown prediction classes and prediction classes with no definition from data source
df.drop_duplicates(inplace=True)
df = df[df.pitch_type != 'UN']
df = df[df.pitch_type != 'FA']

features = ['break_angle', 'break_length', 'end_speed', 'start_speed', 'spin_rate', 'pitch_num',
           'ax', 'ay', 'az', 'nasty', 'sz_bot', 'sz_top', 'vx0', 'vy0', 'vz0', 'x', 'y']
categorical_features = ['on_1b', 'on_2b', 'on_3b', 'outs', 's_count', 'b_count']

#remove wild pitches or intentional walks as break length outliers. Code sourced from https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame
for feature in features:
    if feature != 'pitch_type':
        df = df[np.abs(df[feature]-df[feature].mean()) <= (12*df[feature].std())]

#drop data points where prediction class is unreliable
df = df[df.type_confidence >= 1.0]

#Drop any data with incomplete records
df.dropna(inplace=True)
#normalize angles from -1 to 1 to avoid neuron saturation
df['break_angle']/=180

In [3]:
print(df.columns)

Index(['ab_id', 'ax', 'ay', 'az', 'b_count', 'b_score', 'break_angle',
       'break_length', 'break_y', 'code', 'end_speed', 'nasty', 'on_1b',
       'on_2b', 'on_3b', 'outs', 'pfx_x', 'pfx_z', 'pitch_num', 'pitch_type',
       'px', 'pz', 's_count', 'spin_dir', 'spin_rate', 'start_speed', 'sz_bot',
       'sz_top', 'type', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0',
       'y', 'y0', 'z0', 'zone'],
      dtype='object')


In [4]:
from sklearn.preprocessing import StandardScaler
import gc
scaler = StandardScaler()
ohe = OneHotEncoder()
X = df
y = pd.get_dummies(df['pitch_type'],prefix='type')


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=None)

X_train_ohe = ohe.fit_transform(X_train[categorical_features])
print(type(X_train_ohe))
X_test_ohe = ohe.transform(X_test[categorical_features].values)

X_train = X_train[features]
X_test = X_test[features]

scaler.fit_transform(X_train)
scaler.fit_transform(X_test)
testing_weights = {}
weights = []
for col in y_test.columns:
    testing_weights[col] = (len(y_train[y_train[col] == 1]) / len(y_train))
print(testing_weights)
for _, row in y_test.iterrows():
    for col in testing_weights.keys():
        if row[col] == 1:
            weights.append(testing_weights[col])
            
            
del testing_weights
gc.collect()

/home/rasimon/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<class 'scipy.sparse.csr.csr_matrix'>


/home/rasimon/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/rasimon/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/rasimon/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/rasimon/.local/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


{'type_CH': 0.10411953294209422, 'type_CU': 0.08297611614476366, 'type_EP': 0.00034722600676078617, 'type_FC': 0.05279392370058841, 'type_FF': 0.35456602198890436, 'type_FO': 0.00034255480487611195, 'type_FS': 0.015596364559279888, 'type_FT': 0.1163876661585437, 'type_KC': 0.024706765301689575, 'type_KN': 0.002610423319885462, 'type_SC': 4.8269086141633955e-05, 'type_SI': 0.08397419628078906, 'type_SL': 0.16153093970568314}


0

The dataset consists 15 prediction classes describing the information of the pitch thrown. The prediction classes are as follows: <br><br>1. FC – Cutter <br> 2. FF – Four Seam Fastball <br> 3. FS – Splitter <br> 4. FT – Two Seam Fastball <br> 5. SI – Sinker <br> 6. CU – Curveball <br> 7. KC – Knuckle Curve <br> 8. KN – Knuckleball <br> 9. SL – Slider <br> 10. SC - Screwball <br> 11. CH – Changeup <br> 12. EP – Eephus <br> 13. FO – Pitchout <br> 14. PO – Pitchout <br> 15. IN – Intentional Ball



# 1.1 Wide and Deep Network

We have used a wide network of 1 fully connected layer with relu activation and a dense nework of 4-layers with ReLu activation and an output layer with signoid activation 

In [5]:
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, concatenate
from keras.models import Model
from sklearn.preprocessing import StandardScaler

inputs = Input(shape=(X_train[features].shape[1],))
inputs_wide = Input(shape=(X_train_ohe.shape[1],))
output_wide = Dense(units=3, activation='relu')(inputs_wide)

x = Dense(units=34, activation='relu')(inputs)
x = Dense(units=27, activation='relu')(x)
x = Dense(units=19, activation='relu')(x)
x = Dense(units=15, activation='relu')(x)
output_deep = Dense(13, activation ='sigmoid')(x)

final_branch = concatenate([output_wide, output_deep], name='concat_1')
final_branch = Dense(units=13, activation='softmax', name = 'combined')(final_branch)

model = Model(inputs = [inputs, inputs_wide], outputs = final_branch)
model.compile(optimizer='adam', loss = 'mean_squared_error', metrics=['accuracy'])
model.summary()



Using TensorFlow backend.
W0414 12:14:04.748447 140476576433984 deprecation.py:237] From /home/rasimon/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 17)           0                                            
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 34)           612         input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 27)           945         dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 19)           532         dense_3[0][0]                    
__________________________________________________________________________________________________
input_2 (I

In [6]:
print(X_train_ohe)

  (0, 13)	1.0
  (0, 9)	1.0
  (0, 8)	1.0
  (0, 4)	1.0
  (0, 2)	1.0
  (0, 0)	1.0
  (1, 12)	1.0
  (1, 9)	1.0
  (1, 7)	1.0
  (1, 4)	1.0
  (1, 2)	1.0
  (1, 1)	1.0
  (2, 14)	1.0
  (2, 11)	1.0
  (2, 7)	1.0
  (2, 4)	1.0
  (2, 2)	1.0
  (2, 0)	1.0
  (3, 12)	1.0
  (3, 10)	1.0
  (3, 8)	1.0
  (3, 4)	1.0
  (3, 2)	1.0
  (3, 0)	1.0
  (4, 12)	1.0
  :	:
  (1284461, 0)	1.0
  (1284462, 14)	1.0
  (1284462, 9)	1.0
  (1284462, 7)	1.0
  (1284462, 4)	1.0
  (1284462, 2)	1.0
  (1284462, 0)	1.0
  (1284463, 14)	1.0
  (1284463, 11)	1.0
  (1284463, 6)	1.0
  (1284463, 4)	1.0
  (1284463, 2)	1.0
  (1284463, 1)	1.0
  (1284464, 13)	1.0
  (1284464, 10)	1.0
  (1284464, 7)	1.0
  (1284464, 4)	1.0
  (1284464, 2)	1.0
  (1284464, 0)	1.0
  (1284465, 15)	1.0
  (1284465, 11)	1.0
  (1284465, 6)	1.0
  (1284465, 4)	1.0
  (1284465, 2)	1.0
  (1284465, 0)	1.0


In [7]:
model.fit([X_train, X_train_ohe] ,y_train, epochs=10, batch_size=50, verbose=1)


Epoch 1/10
1284466/1284466 [==============================] - 115s 90us/step - loss: 0.0353 - acc: 0.6627
Epoch 2/10
1284466/1284466 [==============================] - 85s 66us/step - loss: 0.0298 - acc: 0.7166
Epoch 3/10
1284466/1284466 [==============================] - 73s 57us/step - loss: 0.0287 - acc: 0.7276
Epoch 4/10
1284466/1284466 [==============================] - 65s 51us/step - loss: 0.0282 - acc: 0.7336
Epoch 5/10
1284466/1284466 [==============================] - 61s 47us/step - loss: 0.0279 - acc: 0.7366
Epoch 6/10
1284466/1284466 [==============================] - 59s 46us/step - loss: 0.0277 - acc: 0.7384
Epoch 7/10
1284466/1284466 [==============================] - 57s 45us/step - loss: 0.0275 - acc: 0.7401
Epoch 8/10
1284466/1284466 [==============================] - 57s 44us/step - loss: 0.0274 - acc: 0.7414
Epoch 9/10
1284466/1284466 [==============================] - 56s 44us/step - loss: 0.0273 - acc: 0.7426
Epoch 10/10
1284466/1284466 [=========================

In [9]:

from sklearn import metrics as mt

yhat_proba = model.predict([X_test, X_test_ohe])
yhat = np.round(yhat_proba)
print(mt.accuracy_score(y_test, yhat,sample_weight=weights))

0.8118712016625669
